In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import sklearn
import time

data = pd.read_parquet('train_data.parquet', engine='pyarrow')

In [2]:
# Полезные частые функции

def metrics(y_true, y_preds):
    rec = sklearn.metrics.recall_score(y_true, y_preds, average=None)
    f1 = sklearn.metrics.f1_score(y_true, y_preds)
    prec = 	sklearn.metrics.precision_score(y_true, y_preds)
    auc = sklearn.metrics.roc_auc_score(y_true, y_preds)
    print(' test-recall:', rec, ' test-f1_score:', f1, ' test-precision_score:', prec, ' test-roc_auc_score:', auc)
    
    


In [3]:
metrics([0, 1, 1, 0, 1, 1, 1], [1, 0, 1, 0, 0, 0, 1])

 test-recall: [0.5 0.4]  test-f1_score: 0.5  test-precision_score: 0.6666666666666666  test-roc_auc_score: 0.44999999999999996


In [3]:

useless_columns = ['id', 'target', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']


y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)
x_data

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,1761,1759,85,105469,191,46,0,6,95,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1761,1759,120,105610,144,71,0,135,135,0,...,0,0,0,0,0,0,0,0,0,0
2,890,1759,141,105227,191,11,0,0,0,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,1599,966,30,102441,191,8,0,0,0,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,1761,1759,85,104006,191,2,0,0,46,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,1761,1759,141,410,191,2,125,290,298,176,...,0,0,0,0,0,0,0,0,0,0
693038,1761,1759,0,170,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
693039,1761,1759,0,168,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0
693040,1761,1759,141,74,191,1,125,290,298,176,...,0,0,0,0,0,51714,0,0,0,0


In [4]:
X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)


In [5]:
# EQual size of targets

print(y_train.value_counts())
print(y_test.value_counts())


sample_size = 17000 # Задайте желаемое количество строк для каждого класса
frames = []
classes = data['target'].unique()

for i in classes:
    if i == 0:  
        g = data[data['target'] == i].sample(sample_size)
    else:
        g = data[data['target'] == i].sample(sample_size)
    frames.append(g)

equally_sampled = pd.concat(frames)
equally_sampled['target'].value_counts()

X_train_equally_sampled, X_test_equally_sampled, y_train_equally_sampled,  y_test_equally_sampled = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)
"""X_train_xgb_equally_sampled = xgb.DMatrix(X_train_equally_sampled, label=y_train_equally_sampled)
X_test_xgb_equally_sampled = xgb.DMatrix(X_test_equally_sampled, label=y_test_equally_sampled)"""

target
0    375794
1     13917
Name: count, dtype: int64
target
0    125284
1      4620
Name: count, dtype: int64


'X_train_xgb_equally_sampled = xgb.DMatrix(X_train_equally_sampled, label=y_train_equally_sampled)\nX_test_xgb_equally_sampled = xgb.DMatrix(X_test_equally_sampled, label=y_test_equally_sampled)'

In [7]:
equally_sampled['target'].value_counts()

target
0    17000
1    17000
Name: count, dtype: int64

In [8]:
xgb_data = xgb.DMatrix(x_data, label=y_data)

In [12]:
params = {'max_depth': 4,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'map']}
eval_list = [(X_test_xgb, "Everything")]
num_rounds = 22

In [9]:
evals_result = dict()

def objective(n_estimators, max_depth, classes_weight_o, learning_rate):
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    classes_weight_o = int(classes_weight_o)
    classes_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:classes_weight_o},
    y=y_train
)   

    
    xgb_model = xgb.XGBClassifier(n_estimators=n_estimators,
                                max_depth=max_depth,
                                learning_rate=0.01,
                                booster="gbtree",
                                n_jobs=-1,
                                eval_metric=['auc', 'pre'],
                                verbose=0)

    xgb_model.fit(X_train, y_train, eval_set=eval_list, sample_weight=classes_weights)

    return sklearn.metrics.roc_auc_score(y_test, xgb_model.predict(X_test))

parameters = {
    'max_depth': [6, 8],
    'learning_rate': [0.2],
    'n_estimators': [20, 40],
}


"""params = {'max_depth': 6,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre'],
          'class_weight': {0:0.1, 1:100}}"""
eval_list = [(x_data, y_data), (X_train, y_train), (X_test, y_test)]

from sklearn.utils import class_weight
"""classes_weights = class_weight.compute_sample_weight(
    class_weight={0:1, 1:22},
    y=y_train
)"""
#clf = sklearn.model_selection.GridSearchCV(xgb_model, parameters, scoring='f1_micro', n_jobs=-1, verbose=10)

search_space = {
    'max_depth': (4, 18),
    'n_estimators': (10, 40),
    "classes_weight_o": (18, 27),
    'learning_rate': (0.009, 0.2)
}
import bayes_opt
optimizer = bayes_opt.BayesianOptimization(
                                f=objective,
                                pbounds=search_space,
                                random_state=123,
                                allow_duplicate_points=True
                              ) 

opt = optimizer.maximize(n_iter=20)

|   iter    |  target   | classe... | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------


C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:40:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.72713	validation_0-pre:0.31250	validation_1-auc:0.73273	validation_1-pre:0.34375	validation_2-auc:0.71027	validation_2-pre:0.34375
[1]	validation_0-auc:0.73010	validation_0-pre:0.31250	validation_1-auc:0.73531	validation_1-pre:0.34375	validation_2-auc:0.71444	validation_2-pre:0.34375
[2]	validation_0-auc:0.73077	validation_0-pre:0.31250	validation_1-auc:0.73645	validation_1-pre:0.34375	validation_2-auc:0.71368	validation_2-pre:0.34375
[3]	validation_0-auc:0.73269	validation_0-pre:0.31250	validation_1-auc:0.73817	validation_1-pre:0.34375	validation_2-auc:0.71621	validation_2-pre:0.34375
[4]	validation_0-auc:0.73655	validation_0-pre:0.40625	validation_1-auc:0.74243	validation_1-pre:0.43750	validation_2-auc:0.71882	validation_2-pre:0.43750
[5]	validation_0-auc:0.74093	validation_0-pre:0.40625	validation_1-auc:0.74697	validation_1-pre:0.46875	validation_2-auc:0.72271	validation_2-pre:0.43750
[6]	validation_0-auc:0.74108	validation_0-pre:0.40625	validation_1-auc:0.747

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [22:43:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.84662	validation_0-pre:0.71875	validation_1-auc:0.93050	validation_1-pre:0.96875	validation_2-auc:0.59431	validation_2-pre:0.12500
[1]	validation_0-auc:0.85349	validation_0-pre:0.71875	validation_1-auc:0.93731	validation_1-pre:0.96875	validation_2-auc:0.59947	validation_2-pre:0.15625
[2]	validation_0-auc:0.85919	validation_0-pre:0.78125	validation_1-auc:0.94443	validation_1-pre:1.00000	validation_2-auc:0.59939	validation_2-pre:0.18750
[3]	validation_0-auc:0.86445	validation_0-pre:0.87500	validation_1-auc:0.94925	validation_1-pre:1.00000	validation_2-auc:0.60474	validation_2-pre:0.06250
[4]	validation_0-auc:0.86946	validation_0-pre:0.87500	validation_1-auc:0.95386	validation_1-pre:1.00000	validation_2-auc:0.60986	validation_2-pre:0.25000
[5]	validation_0-auc:0.87227	validation_0-pre:0.90625	validation_1-auc:0.95683	validation_1-pre:1.00000	validation_2-auc:0.61158	validation_2-pre:0.25000
[6]	validation_0-auc:0.87736	validation_0-pre:0.90625	validation_1-auc:0.960

...

[0]	validation_0-auc:0.73553	validation_0-pre:0.46875	validation_1-auc:0.74713	validation_1-pre:0.46875	validation_2-auc:0.70054	validation_2-pre:0.34375
[1]	validation_0-auc:0.74124	validation_0-pre:0.40625	validation_1-auc:0.75323	validation_1-pre:0.40625	validation_2-auc:0.70509	validation_2-pre:0.34375
[2]	validation_0-auc:0.74281	validation_0-pre:0.40625	validation_1-auc:0.75493	validation_1-pre:0.40625	validation_2-auc:0.70625	validation_2-pre:0.34375
[3]	validation_0-auc:0.74503	validation_0-pre:0.40625	validation_1-auc:0.75717	validation_1-pre:0.40625	validation_2-auc:0.70839	validation_2-pre:0.34375
[4]	validation_0-auc:0.74565	validation_0-pre:0.40625	validation_1-auc:0.75775	validation_1-pre:0.40625	validation_2-auc:0.70912	validation_2-pre:0.34375
[5]	validation_0-auc:0.75178	validation_0-pre:0.34375	validation_1-auc:0.76420	validation_1-pre:0.34375	validation_2-auc:0.71429	validation_2-pre:0.31250
[6]	validation_0-auc:0.75261	validation_0-pre:0.34375	validation_1-auc:0.764

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:03:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.69575	validation_0-pre:0.21875	validation_1-auc:0.69598	validation_1-pre:0.37500	validation_2-auc:0.69505	validation_2-pre:0.28125
[1]	validation_0-auc:0.69596	validation_0-pre:0.21875	validation_1-auc:0.69616	validation_1-pre:0.37500	validation_2-auc:0.69535	validation_2-pre:0.28125
[2]	validation_0-auc:0.70283	validation_0-pre:0.21875	validation_1-auc:0.70320	validation_1-pre:0.37500	validation_2-auc:0.70173	validation_2-pre:0.28125
[3]	validation_0-auc:0.70917	validation_0-pre:0.21875	validation_1-auc:0.70908	validation_1-pre:0.37500	validation_2-auc:0.70946	validation_2-pre:0.28125
[4]	validation_0-auc:0.70877	validation_0-pre:0.28125	validation_1-auc:0.70873	validation_1-pre:0.25000	validation_2-auc:0.70890	validation_2-pre:0.31250
[5]	validation_0-auc:0.71080	validation_0-pre:0.28125	validation_1-auc:0.71073	validation_1-pre:0.25000	validation_2-auc:0.71100	validation_2-pre:0.31250
[6]	validation_0-auc:0.71157	validation_0-pre:0.28125	validation_1-auc:0.711

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:04:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.69540	validation_0-pre:0.21875	validation_1-auc:0.69462	validation_1-pre:0.34375	validation_2-auc:0.69774	validation_2-pre:0.28125
[1]	validation_0-auc:0.69579	validation_0-pre:0.21875	validation_1-auc:0.69502	validation_1-pre:0.34375	validation_2-auc:0.69811	validation_2-pre:0.28125
[2]	validation_0-auc:0.69612	validation_0-pre:0.25000	validation_1-auc:0.69538	validation_1-pre:0.34375	validation_2-auc:0.69834	validation_2-pre:0.28125
[3]	validation_0-auc:0.69634	validation_0-pre:0.25000	validation_1-auc:0.69554	validation_1-pre:0.34375	validation_2-auc:0.69873	validation_2-pre:0.28125
[4]	validation_0-auc:0.69666	validation_0-pre:0.25000	validation_1-auc:0.69590	validation_1-pre:0.34375	validation_2-auc:0.69894	validation_2-pre:0.28125
[5]	validation_0-auc:0.70505	validation_0-pre:0.25000	validation_1-auc:0.70440	validation_1-pre:0.34375	validation_2-auc:0.70700	validation_2-pre:0.28125
[6]	validation_0-auc:0.70597	validation_0-pre:0.25000	validation_1-auc:0.705

In [15]:
opt

In [18]:

y_preds = xgb_model.predict(X_test)
y_true = list(y_test)
print(*y_preds)

#ev = clf.eval(X_test)
metrics(y_true, y_preds)
#print(ev, ' eval-recall:')

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_preds, y_test))
cm = confusion_matrix(y_preds, y_test)
acc = cm.diagonal().sum()/cm.sum()
print(acc)

0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

# GRIDSEARCH XGBOOST

In [9]:
params = {
    'max_depth': range(2,10,2),
    'learning_rate': np.linspace(.1, .6, 6),
    'min_child_weight': range(1,10,2),
}

grid = sklearn.model_selection.GridSearchCV(
    estimator = xgb.XGBClassifier(n_jobs=-1,
                              n_estimators=500,
                              random_state=0),
    param_grid = params,
    scoring='f1',  # <------Use `scoring` instead of `eval_metric`
)

eval_set = [(X_train_equally_sampled, y_train_equally_sampled),
            (X_test_equally_sampled, y_test_equally_sampled)]

grid.fit(X_train_equally_sampled, y_train_equally_sampled,
         eval_set=eval_set,
         early_stopping_rounds=25,
)

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.20259	validation_1-logloss:0.20320
[1]	validation_0-logloss:0.19513	validation_1-logloss:0.19577
[2]	validation_0-logloss:0.18857	validation_1-logloss:0.18927
[3]	validation_0-logloss:0.18285	validation_1-logloss:0.18356
[4]	validation_0-logloss:0.17785	validation_1-logloss:0.17859
[5]	validation_0-logloss:0.17346	validation_1-logloss:0.17426
[6]	validation_0-logloss:0.16963	validation_1-logloss:0.17045
[7]	validation_0-logloss:0.16629	validation_1-logloss:0.16714
[8]	validation_0-logloss:0.16335	validation_1-logloss:0.16425
[9]	validation_0-logloss:0.16079	validation_1-logloss:0.16171
[10]	validation_0-logloss:0.15856	validation_1-logloss:0.15950
[11]	validation_0-logloss:0.15660	validation_1-logloss:0.15756
[12]	validation_0-logloss:0.15490	validation_1-logloss:0.15589
[13]	validation_0-logloss:0.15341	validation_1-logloss:0.15442
[14]	validation_0-logloss:0.15208	validation_1-logloss:0.15314
[15]	validation_0-logloss:0.15095	validation_1-logloss:0.15202
[1

KeyboardInterrupt: 

In [ ]:
data

,id,target,sample_ml_new,feature1,feature2,feature3,feature4,feature5,feature6,feature7,...,feature1067,feature1068,feature1069,feature1070,feature1071,feature1072,feature1073,feature1074,feature1075,feature1076
0,0,0,1,1761,1759,85,105469,191,46,0,...,28913,48985,84264,12045,12107,12510,21126,28913,48985,84264
1,1,0,1,1761,1759,120,105610,144,71,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,890,1759,141,105227,191,11,0,...,55949,70006,113317,33735,31242,12691,48682,55949,70006,113317
5,5,0,1,1599,966,30,102441,191,8,0,...,0,6872,8530,0,0,0,0,0,6872,8530
6,6,0,1,1761,1759,85,104006,191,2,0,...,0,0,7642,0,0,0,0,0,0,7642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693036,693036,0,1,1761,1759,141,410,191,2,125,...,0,0,0,0,0,0,0,0,0,0
693038,693038,0,1,1761,1759,0,170,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693039,693039,0,1,1761,1759,0,168,191,1,125,...,0,0,0,0,0,51714,0,0,0,0
693040,693040,0,1,1761,1759,141,74,191,1,125,...,0,0,0,0,0,51714,0,0,0,0


In [67]:
data['feature5'].describe()

count    519615.000000
mean        139.003685
std          74.734122
min           0.000000
25%          63.000000
50%         191.000000
75%         191.000000
max         191.000000
Name: feature5, dtype: float64

In [159]:
data['feature2'].value_counts()

feature2
1759    334027
0         4505
1         3242
2         2660
3         2174
         ...  
1177         7
1708         7
1711         6
1717         6
1518         5
Name: count, Length: 1728, dtype: int64

In [15]:
x=140
corr = data[['target', 'feature'+str(x), 'feature'+str(x+1), 'feature'+str(x+2), 'feature'+str(x+3), 
                'feature'+str(x+4), 'feature'+str(x+5), 'feature'+str(x+6), 'feature'+str(x+7), 'feature'+str(x+8),
                'feature'+str(x+9)]].corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\kseni\anaconda3\Lib\site-packages\pandas\io\formats\style.py:3618: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
C:\Users\kseni\anaconda3\Lib\site-packages\pandas\io\formats\style.py:3619: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,target,feature140,feature141,feature142,feature143,feature144,feature145,feature146,feature147,feature148,feature149
target,1.000000,0.018013,-0.027897,-0.035972,0.003920,nan,-0.003309,-0.005981,-0.009559,0.010159,0.016216
feature140,0.018013,1.000000,0.074863,0.077542,0.072244,nan,0.089497,-0.003397,0.086332,0.020580,0.032358
feature141,-0.027897,0.074863,1.000000,0.493605,0.122855,nan,0.301292,-0.009977,0.312402,-0.034708,0.014048
feature142,-0.035972,0.077542,0.493605,1.000000,0.154144,nan,0.564539,-0.017306,0.655718,-0.089917,-0.060648
feature143,0.003920,0.072244,0.122855,0.154144,1.000000,nan,0.144641,-0.001349,0.149737,0.016801,0.045139
feature144,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
feature145,-0.003309,0.089497,0.301292,0.564539,0.144641,nan,1.000000,-0.016940,0.886586,0.020851,-0.000943
feature146,-0.005981,-0.003397,-0.009977,-0.017306,-0.001349,nan,-0.016940,1.000000,-0.014700,0.024892,0.005746
feature147,-0.009559,0.086332,0.312402,0.655718,0.149737,nan,0.886586,-0.014700,1.000000,-0.054621,0.002148
feature148,0.010159,0.020580,-0.034708,-0.089917,0.016801,nan,0.020851,0.024892,-0.054621,1.000000,0.026684


In [68]:
boxplot = data.boxplot("feature5")

In [7]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
import xgboost as xgb

useless_columns = ['id', 'target', 'sample_ml_new', 'feature68', 'feature69', 'feature144', 'feature160',
                   'feature292', 'feature406', 'feature407', 'feature496', 'feature511',
                   'feature625', 'feature661', 'feature663', 'feature669', 'feature678',
                   'feature683', 'feature686', 'feature710', 'feature756', 'feature761',
                   'feature765', 'feature801', 'feature802', 'feature806', 'feature807',
                   'feature808', 'feature809', 'feature816', 'feature818', 'feature819',
                   'feature955', 'feature956', 'feature957', 'feature958', 'feature959',
                   'feature960', 'feature961', 'feature962', 'feature963', 'feature964',
                   'feature965', 'feature966', 'feature967', 'feature968', 'feature969',
                   'feature970', 'feature971', 'feature972', 'feature973', 'feature974', 
                   'feature975', 'feature976', 'feature977', 'feature978', 'feature979',
                   'feature980', 'feature981', 'feature982', 'feature983', 'feature984', 
                   'feature1005', 'feature1006', 'feature1007', 'feature1008', 'feature1009',
                   'feature1010', 'feature1011', 'feature1012', 'feature1013', 'feature1014',
                   'feature1015', 'feature1016', 'feature1017', 'feature1018', 'feature1019',
                   'feature1020', 'feature1021', 'feature1022', 'feature1023', 'feature1024',
                   'feature1025', 'feature1026', 'feature1027', 'feature1028', 'feature1029',
                   'feature1030', 'feature1031', 'feature1032', 'feature1033', 'feature1034']

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]

x_data = data.drop(useless_columns, axis=1)
print(x_data.columns)

Index(['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 'feature6',
       'feature7', 'feature8', 'feature9', 'feature10',
       ...
       'feature1067', 'feature1068', 'feature1069', 'feature1070',
       'feature1071', 'feature1072', 'feature1073', 'feature1074',
       'feature1075', 'feature1076'],
      dtype='object', length=987)


In [8]:
X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

X_train_xgb = xgb.DMatrix(X_train, label=y_train)
X_test_xgb = xgb.DMatrix(X_test, label=y_test)

In [9]:
from sklearn.impute import SimpleImputer

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns


cols_with_missing = [col for col in train_num_cleaned.columns
                                if test_num_cleaned[col].isnull().any()]
print(cols_with_missing)
for col in cols_with_missing:
    print(col, test_num_cleaned[col].isnull().any())

[]


In [12]:
# Изменение веса классов

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_data), y=y_data)

# Создание модели с взвешиванием классов
model = RandomForestClassifier(n_estimators=30, class_weight={0: 0.5, 1: 500.0})
model.fit(train_num_cleaned, y_train)

RandomForestClassifier(class_weight={0: 0.5, 1: 100.0}, n_estimators=30)

In [14]:
preds = model.predict(test_num_cleaned)
preds = list(map(round, preds))
y_true = list(y_test)
print(preds.count(1))
#ev = bst.eval(X_test_xgb)


rec = sklearn.metrics.recall_score(y_true, preds)
f1 = sklearn.metrics.f1_score(y_true, preds)
prec = 	sklearn.metrics.precision_score(y_true, preds)
auc = sklearn.metrics.roc_auc_score(y_true, preds)
print(' test-recall:', rec, ' test-f1_score:', f1, ' test-precision_score:', prec, ' test-roc_auc_score:', auc)

21
 test-recall: 0.0012942191544434857  test-f1_score: 0.0025767661584711185  test-precision_score: 0.2857142857142857  test-roc_auc_score: 0.5005872379420077


In [15]:
# Методы борьбы с дисбалансом данных
import sklearn
print(sklearn.__version__)
import pandas as pd
from imblearn.over_sampling import SMOTE

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

# Создание экземпляра SMOTE
smote = SMOTE()

# Применение SMOTE к данным
X_resampled, y_resampled = smote.fit_resample(train_num_cleaned, y_train)

X_resampled_xgb = xgb.DMatrix(X_resampled, y_resampled)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_test)


1.2.1


In [22]:
params = {'max_depth': 8,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre']}
eval_list = [(X_test_xgb, "Everything")]
num_rounds = 80

In [23]:
y_resampled.value_counts()

target
0    375812
1    375812
Name: count, dtype: int64

In [24]:
evals_result = dict()
bst = xgb.train(params, X_resampled_xgb, num_rounds, eval_list, evals=X_test_xgb, evals_result=evals_result)
y_preds = bst.predict(X_test_xgb)
preds = list(map(round, preds))
y_true = list(y_test)
metrics(y_true, preds)

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	Everything-auc:0.65567	Everything-pre:0.12500
[1]	Everything-auc:0.64536	Everything-pre:0.18750
[2]	Everything-auc:0.66112	Everything-pre:0.12500
[3]	Everything-auc:0.67545	Everything-pre:0.31250
[4]	Everything-auc:0.68615	Everything-pre:0.25000
[5]	Everything-auc:0.68696	Everything-pre:0.15625
[6]	Everything-auc:0.68630	Everything-pre:0.12500
[7]	Everything-auc:0.69125	Everything-pre:0.12500
[8]	Everything-auc:0.69041	Everything-pre:0.18750
[9]	Everything-auc:0.68784	Everything-pre:0.18750
[10]	Everything-auc:0.68277	Everything-pre:0.18750
[11]	Everything-auc:0.68333	Everything-pre:0.18750
[12]	Everything-auc:0.68161	Everything-pre:0.15625
[13]	Everything-auc:0.67953	Everything-pre:0.18750
[14]	Everything-auc:0.67673	Everything-pre:0.21875
[15]	Everything-auc:0.67283	Everything-pre:0.18750
[16]	Everything-auc:0.67126	Everything-pre:0.15625
[17]	Everything-auc:0.66832	Everything-pre:0.18750
[18]	Everything-auc:0.66646	Everything-pre:0.18750
[19]	Everything-auc:0.66607	Everything-pr

In [26]:
metrics(y_true, preds)

 test-recall: 0.0  test-f1_score: 0.0  test-precision_score: 0.0  test-roc_auc_score: 0.49991617837242347


In [27]:
# Обучение каждой моделей члена на подмножестве данных

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

X_train_xgb = xgb.DMatrix(train_num_cleaned, y_train)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_train)

eval_list = [(X_test_xgb, "Everything")]
num_rounds = 40
ensemble = [xgb.train({'max_depth': x,
          'eta': 1,
          'objective': "binary:logistic",
          'nthread': -1,
          'eval_metric': ['auc', 'pre']}, X_train_xgb, num_rounds, eval_list, evals=X_test_xgb, evals_result=evals_result) for x in range(2, 10)]
ind = 1

"""for model in ensemble:
    sample_indices = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_subset, y_subset = X_train[sample_indices], y_train[sample_indices]
    model.fit(X_subset, y_subset)
    print(f"Модель {ind} обучена")
    ind +=1"""

C:\Users\kseni\anaconda3\Lib\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	Everything-auc:0.59881	Everything-pre:0.06250
[1]	Everything-auc:0.61184	Everything-pre:0.00000
[2]	Everything-auc:0.61897	Everything-pre:0.06250
[3]	Everything-auc:0.61701	Everything-pre:0.03125
[4]	Everything-auc:0.61996	Everything-pre:0.06250
[5]	Everything-auc:0.61903	Everything-pre:0.06250
[6]	Everything-auc:0.65060	Everything-pre:0.06250
[7]	Everything-auc:0.65347	Everything-pre:0.09375
[8]	Everything-auc:0.64697	Everything-pre:0.12500
[9]	Everything-auc:0.64366	Everything-pre:0.12500
[10]	Everything-auc:0.64874	Everything-pre:0.09375
[11]	Everything-auc:0.64773	Everything-pre:0.03125
[12]	Everything-auc:0.65129	Everything-pre:0.06250
[13]	Everything-auc:0.65373	Everything-pre:0.15625
[14]	Everything-auc:0.65100	Everything-pre:0.09375
[15]	Everything-auc:0.64985	Everything-pre:0.06250
[16]	Everything-auc:0.65037	Everything-pre:0.09375
[17]	Everything-auc:0.65434	Everything-pre:0.09375
[18]	Everything-auc:0.64982	Everything-pre:0.15625
[19]	Everything-auc:0.65507	Everything-pr

'for model in ensemble:\n    sample_indices = np.random.choice(len(X_train), size=len(X_train), replace=True)\n    X_subset, y_subset = X_train[sample_indices], y_train[sample_indices]\n    model.fit(X_subset, y_subset)\n    print(f"Модель {ind} обучена")\n    ind +=1'

In [49]:
from statistics import mode
predictions = np.array([model.predict(X_test_xgb) for model in ensemble])
new_predictions=[]
for x in range(len(predictions[0])):
    new_predictions.append(np.array([predictions[y][x] for y in range(len(predictions))]))
ensemble_predictions = [int(mode(x.round())) for x in new_predictions]
# Используем np.argmax для нахождения индекса наиболее часто встречающегося значения

metrics(y_test, ensemble_predictions)

 test-recall: 0.004959034066407934  test-f1_score: 0.009799744354495099  test-precision_score: 0.4107142857142857  test-roc_auc_score: 0.5023477973327265


In [ ]:
# Кросс-обучение

data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

X_train_xgb = xgb.DMatrix(train_num_cleaned, y_train)
X_test_xgb = xgb.DMatrix(test_num_cleaned, y_train)

In [ ]:
data = pd.read_parquet('train_data.parquet', engine='pyarrow')
y_data = data["target"]
x_data = data.drop(useless_columns, axis=1)

from sklearn.impute import SimpleImputer


X_train, X_test, y_train,  y_test = sklearn.model_selection.train_test_split(x_data, y_data, test_size=0.25, shuffle=True)

# impute
train_data_num = X_train.select_dtypes(exclude=['object'])
test_data_num = X_test.select_dtypes(exclude=['object'])
imputer = SimpleImputer()
train_num_cleaned = imputer.fit_transform(train_data_num)
test_num_cleaned = imputer.transform(test_data_num)

# columns rename after imputing
train_num_cleaned = pd.DataFrame(train_num_cleaned)
test_num_cleaned = pd.DataFrame(test_num_cleaned)
train_num_cleaned.columns = train_data_num.columns
test_num_cleaned.columns = test_data_num.columns

